In [3]:
import sys
sys.path.append('../')
import numpy as np
import os
from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool, cpu_count
from tilted_perovskite_simulation.perovskite import Perovskite
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate CIF data

In [3]:
def write_single_cif(args):
    tilt_angles, lattice_param, atom_species, uiso, foldername, filename = args
    perovskite = Perovskite(tilt_angles=tilt_angles, lattice_param=lattice_param, atom_species=atom_species, uiso=uiso)
    basename = '{}_{:.2f}_{:.2f}_{:.2f}'.format(
        filename, tilt_angles[0], tilt_angles[1], tilt_angles[2])
    cif_filedir = foldername + "/" + basename + ".cif"
    perovskite.save_structure_to_cif(cif_filedir)
    return [basename, tilt_angles[0], tilt_angles[1], tilt_angles[2]]
    
def construct_tasks(num_samples: int, 
                    tilt_range: float,
                    lattice_param: float,
                    atom_species: list,
                    uiso: list,
                    foldername: str,
                    filename: str,
                    random_seed: int = 0,
                    ):
    np.random.seed(random_seed)
    if not os.path.exists(foldername):
        os.makedirs(foldername)
    tasks = []
    for _ in range(num_samples):
        tilt_angles = np.random.uniform(-tilt_range, tilt_range, size=3)
        tasks.append((tilt_angles, lattice_param, atom_species, uiso, foldername, filename))
    return tasks

In [4]:
num_samples = 100000 # 100,000
tilt_range = 15
lattice_param = 4.0  # lattice parameter in Å
atom_species = ["Ca", "Ti", "O"]  # in the order of ABX
uiso = np.array([0.005, 0.003, 0.01])  # in the order of ABX
foldername = "../data/cif"
filename = 'Perov'

tasks = construct_tasks(num_samples, tilt_range, lattice_param, atom_species, uiso, foldername, filename)
with Pool(processes=cpu_count()) as pool:
    data_summary = list(tqdm(pool.imap(write_single_cif, tasks), total=len(tasks)))
np.savetxt(f'../data/data_summary.csv', np.array(data_summary), delimiter=',', fmt='%s')

100%|██████████| 100000/100000 [02:15<00:00, 738.22it/s]


# Generate PDF data

In [2]:
import sys
sys.path.append('../')
import numpy as np
import os
import glob
from pathlib import Path
from diffpy.Structure import loadStructure
from diffpy.srreal.pdfcalculator import PDFCalculator
from pathlib import Path
from sympy import *
from typing import *
from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool, cpu_count
from tilted_perovskite_simulation.data_utils import write_single_pdf
from tilted_perovskite_simulation.perovskite import Perovskite
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [6]:
def construct_tasks(cif_folder_dir: str, pdf_folder_dir: str, pc_cfg: dict):
    if not os.path.exists(pdf_folder_dir):
        os.makedirs(pdf_folder_dir)
    tasks = []
    for cif_file_dir in glob.glob("{}/*.cif".format(cif_folder_dir)):
        cif_file_name = os.path.basename(cif_file_dir)
        pdf_file_dir = Path(pdf_folder_dir, cif_file_name).with_suffix(".gr")
        tasks.append((cif_file_dir, pdf_file_dir, pc_cfg))
    return tasks

In [7]:
pc_cfg = {
    "rmax": 30,
    "rstep": 0.01,
    "qmin": 0.6,
    "qmax": 24,
    "qdamp": 0.03,
    "qbroad": 0.01,
    "delta2": 2.1,
}

cif_folder_dir = '../data/cif'
pdf_folder_dir = '../data/pdf'

mp.set_start_method('spawn', force=True)
tasks = construct_tasks(cif_folder_dir, pdf_folder_dir, pc_cfg)
with Pool(processes=cpu_count()) as pool:
    _ = list(tqdm(pool.imap(write_single_pdf, tasks), total=len(tasks)))

100%|██████████| 100000/100000 [1:05:58<00:00, 25.26it/s]


# Experimental Data

### My Prediction

In [ ]:
tilt_angles = [-12.223058,  -5.520658, 17.591356]
lattice_param = 3.91
atom_species = ["Ca", "Ti", "O"]
uiso = [5.18344632e-03, 2.69926865e-03, 1.04105649e-02]
perovskite = Perovskite(tilt_angles=tilt_angles, lattice_param=lattice_param, atom_species=atom_species, uiso=uiso)
perovskite.save_structure_to_cif('../Perov_-12.223_-5.521_17.591.cif')

### Sandra's Prediction

In [5]:
tilt_angles = [-9.7, -9.7, 7.6]
lattice_param = 3.91
atom_species = ["Ca", "Ti", "O"]
uiso = [5.18344632e-03, 2.69926865e-03, 1.04105649e-02]
perovskite = Perovskite(tilt_angles=tilt_angles, lattice_param=lattice_param, atom_species=atom_species, uiso=uiso)
perovskite.save_structure_to_cif('../Perov_-9.7_-9.7_7.6.cif')

### Generate PDF

In [6]:
from tilted_perovskite_simulation.data_utils import write_multiple_pdf
pc_cfg = {
    "rmax": 30,
    "rstep": 0.01,
    "qmin": 0.6,
    "qmax": 23.6,
    "qdamp": 0.0291601718936,
    "qbroad": 0.0103215402787,
    "delta2": 2.1261,
}
write_multiple_pdf('../', '../', pc_cfg)